## Import Required Libraries

In [ ]:
# Download from the internet if not already installed
import sys
!{sys.executable} -m pip install numpy > /dev/null
!{sys.executable} -m pip install matplotlib > /dev/null
!{sys.executable} -m pip install pandas > /dev/null
# download tensorflow-gpu instead if you have a graphics card
# go to: https://www.tensorflow.org/install/gpu
!{sys.executable} -m pip install tensorflow > /dev/null
!{sys.executable} -m pip install fastai > /dev/null

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, losses
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Importing and processing our data

In [ ]:
PATH = '../data/'

train_dir = os.path.join(PATH, 'train')
test_dir = os.path.join(PATH, 'test')

train_benign_dir = os.path.join(train_dir, 'benign')
train_malignant_dir = os.path.join(train_dir, 'malignant')
test_benign_dir = os.path.join(test_dir, 'benign')
test_malignant_dir = os.path.join(test_dir, 'malignant')

In [ ]:
train_image_generator = ImageDataGenerator(rescale=1./255)
test_image_generator = ImageDataGenerator(rescale=1./255)

train_data_gen = train_image_generator.flow_from_directory(batch_size=128,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(224, 224),
                                                           class_mode='binary')

test_data_gen = test_image_generator.flow_from_directory(batch_size=128,
                                                         directory=test_dir,
                                                         target_size=(224, 224),
                                                         class_mode='binary')

### Showcase of some images

## Creating a Machine Learning Model with Keras

## Results